<a href="https://colab.research.google.com/github/yashmakwana0702/Programing/blob/master/IR_week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import csv
import gc
import sys
csv.field_size_limit(sys.maxsize) # if we don't do this, we won't be able to read whole line ( try to comment this line for action )

131072

In [ ]:
drive.mount('/content/drive')
# going to our folder
% cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
word_doc_index = {} # took around 15/20 secs (altho this will be in our RAM now FYI)
# Storing this in RAM will be heavy, so you have 2 ways, you can either iterate through file, or use RAM
# Storing in RAM 
with open('./IROUTPUTS_v2/merged.txt') as f:
  next(f) # I have one empty line
  for line in f:
    word, *docs = line.strip().split(' ')
    word_doc_index[word] = tuple(docs) # tuple saves a lil bit of memory since its read only

In [ ]:
del word_doc_index
gc.collect()


1180

In [ ]:
doc_id_to_title = {}
shelf_to_title = defaultdict(list)
with open('./gutenberg_data.csv') as f:
  next(f)
  csv_file = csv.reader(f)
  for line in csv_file:
    title, author, link, id, bookshelf, text = line
    doc_id_to_title[id] = title

with open('./gutenberg_metadata.csv') as f:
  next(f)
  csv_file = csv.reader(f)
  for line in csv_file:
    title, author, link, bookshelf = line
    shelf_to_title[bookshelf].append(title)

In [ ]:
# doc_id_to_title
# shelf_to_title

In [ ]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()
def preprocess_word(word):
  word = word.lower()
  return remove_symbols(porter.stem(word))
preprocess_word('happily')

'happili'

In [ ]:
# so I'm just gonna make a simple query for now. we can move on to advanced at later stage
# we also have meta data to use, we can similarly make index files for them as well
def query(comes_with=[], doesnt_come_with=[], shelf=None):
  # comes_with = [porter.stem(i) for i in comes_with] # as we have stemmed words inside the index
  # doesnt_come_with = [porter.stem(i) for i in doesnt_come_with]
  all_docs = set()
  for word in comes_with:
    word = preprocess_word(word)
    docs = word_doc_index.get(word, [])
    # below line can check if it has atleast one word, then accept the doc
    # all_docs.update(set(docs)) # <= this line can work for unioning
    if all_docs:
      all_docs.intersection_update(set(docs)) # this operation will ensure that we want the doc to HAVE ALL THE WORDS
    else:
      all_docs.update(set(docs)) # in case we don't have anything in common
    if not all_docs:
      break
    # moreover we can even break if first for first N words there are no docs in common
  
  negation_docs = set()
  for word in doesnt_come_with:
    word = preprocess_word(word)
    docs = word_doc_index.get(word, [])
    negation_docs.update(set(docs))
  # removing the docs which have all terms which we dont want
  if negation_docs:
    all_docs.difference_update(set(docs)) # remove docs if they contain this word

  doc_titles = [doc_id_to_title[i] for i in all_docs]
  
  if shelf:
    shelf_docs = shelf_to_title.get(shelf, [])
    filtered_docs = [i for i in doc_titles if i in shelf_docs]
    return filtered_docs
  
  return doc_titles
  # above step highly depends on what we are doing with word while storing
# moreover we can do mapping on these doc_ids too
query(comes_with=['fox'], shelf='Animal')

['Fifty Years a Hunter and Trapper',
 'Antarctic Penguins: A Study of Their Social Habits',
 'Wild Animals at Home',
 'Natural History of the Mammalia of India and Ceylon',
 'Deadfalls and Snares',
 'The Bird Book',
 'Animals of the Past',
 "Wolf and Coyote Trapping: An Up-to-Date Wolf Hunter's Guide",
 'Birds, Illustrated',
 'Hunting in Many Lands: The Book of the Boone and Crockett Club',
 'Birds from North Borneo',
 'Birds and Man',
 'What Bird is That?',
 'The Extermination of the American Bison',
 'Zoological Mythology; or, The Legends of Animals, Volume 1 (of 2)',
 'Fox Trapping: A Book of Instruction Telling How to Trap, Snare, Poison and Shoot',
 'Science of Trapping',
 'Our Vanishing Wild Life: Its Extermination and Preservation',
 'Sketches of the Natural History of Ceylon',
 'Artistic Anatomy of Animals',
 'Butterflies and Moths (British)']